In [1]:
import pandas as pd
import numpy as np
import wget
import os
from datetime import datetime
import yfinance as yf
import requests

In [2]:
def crip_quotation(symbol, ini_date, comparison_symbol='BRL', all_data=True, limit=1, aggregate=1, exchange=''):
    from datetime import datetime
    url = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    if exchange:
        url += '&e={}'.format(exchange)
    if all_data:
        url += '&allData=true'
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.fromtimestamp(d) for d in df.time]
    
    df['timestamp'] = df['timestamp'].dt.normalize()
    df = df.set_index('timestamp').drop(['time','high','low','open','volumefrom','volumeto','conversionType','conversionSymbol'],axis=1)
    df = df[ini_date:]
    
    return df

In [3]:
def precos_rf():
    df = pd.DataFrame()
    rf = pd.read_excel('./pesos/rf.xlsx')
    for i in range(0,len(rf)):
        link = rf['download'][i] # link para download do arquivo de rf
        arquivo = rf['arquivo'][i]
        aba = rf['ativo'][i] # o nome da aba é o nome do ativo
        ini_date = rf['data ini'][i]
        # baixa dados
        wget.download(link)
        td = pd.read_excel(arquivo, aba,skiprows=[0]).set_index('Dia') # skiprows é uma lista de linhas para pular. No caso pula a linha 0
        os.remove(arquivo) # deleta depois de baixar
        td = td[ini_date:] # filtra só as linhas a partir da data desejada
        # transforma pro formato datetime
        td = td.set_index(pd.DataFrame([datetime.strptime(date, '%d/%m/%Y').date() for date in td.index[:]])[0]) 
        td = td['PU Compra Manhã']  # filtra a coluna de preço correta
        df[aba] = td # cria uma coluna com o nome do ativo e o preço histórico
    return(df)
precos_rf().head()

100% [..............................................................................] 91136 / 91136

,LTN 010125,LFT 010327,NTN-B Princ 150535
0,,,
2022-05-02,735.84,11503.77,1930.89
2022-05-03,734.44,11509.38,1924.80
2022-05-04,733.90,11515.28,1906.99
2022-05-05,735.98,11521.80,1933.96
2022-05-06,731.99,11528.71,1914.91


In [16]:
def precos_rv(asset_sheet):
    df = pd.DataFrame()
    for i in asset_sheet.index:
        if i == 'caixa':
            df[i] = 1

        elif "BTC" in asset_sheet.index: # se tiver BTC no index ele sabe que é criptomoeda, aí puxa cotacao de outro lugar
            df[i] = crip_quotation(i, asset_sheet['data ini'][0])['close']
        else:
            df[i] = yf.download(i, asset_sheet['data ini'][0], asset_sheet['data fim'][0])['Adj Close']
            # data fim não é baixada. Só baixa até 1 dia antes
    return(df)
#testar:
#asset_sheet = pd.read_excel(f"./pesos/acao.xlsx", sheet_name="Planilha1").set_index("ativo")
#precos_rv(asset_sheet)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,VALE3.SA,KLBN11.SA,MOVI3.SA,PNVL3.SA,RRRP3.SA,TTEN3.SA,LAVV3.SA,ALSO3.SA,BPAC11.SA,YDUQ3.SA,TUPY3.SA,MGLU3.SA,XPBR31.SA,caixa
Date,,,,,,,,,,,,,,
2022-05-26,84.250000,22.170000,17.155163,11.86,48.180000,9.08,5.28,18.850000,24.950001,18.059999,22.750000,4.07,107.930000,1
2022-05-27,85.720001,22.200001,17.896740,12.23,48.959999,9.15,5.33,19.209999,25.200001,17.250000,22.530001,4.00,110.019997,1
2022-05-30,86.650002,22.129999,17.461679,11.64,49.310001,9.41,5.28,19.219999,24.900000,16.540001,22.420000,3.84,109.900002,1


In [5]:
# Cria lista com os nomes dos ativos
abas = pd.ExcelFile(f"./pesos/pesos.xlsx").sheet_names # pega a lista de abas na planilha
lista_abas = [aba for aba in abas if aba.startswith('Planilha')]
lista_ativos = list(pd.read_excel('./pesos/pesos.xlsx', sheet_name=lista_abas[-1])["ativo"]) # pega a lista sempre da aba mais recente

In [6]:
# baixa dados, calcula a cota diária e põe no dataframe "dados"

#processa primeiro renda fixa
if "rf" in lista_ativos:
    df = precos_rf() # pega os preços dos titulos de rf
    rf = pd.read_excel('./pesos/rf.xlsx') 
    pesos = rf.set_index('ativo')['peso']
    df = (df/df.iloc[0,:]*pesos*100).sum(axis=1) # calcula cotização diária da carteira de rf
    dados = pd.DataFrame(df).rename({0:"rf"},axis=1)
    lista_ativos.remove("rf") # tira rf da lista

# processa renda variavel
for ativo in lista_ativos:
    df = pd.DataFrame()
    y = pd.DataFrame()
    precos = pd.DataFrame()
    abas = pd.ExcelFile(f"./pesos/{ativo}.xlsx").sheet_names # pega a lista de abas na planilha
    lista_abas = [aba for aba in abas if aba.startswith('Planilha')]
    lista_abas # cria uma lista com os nomes das abas que começam com "Planilha"
    for aba in lista_abas:
            asset_sheet = pd.read_excel(f"./pesos/{ativo}.xlsx", sheet_name=aba).set_index("ativo")
            asset_weights = asset_sheet['peso']
            df = precos_rv(asset_sheet) # pega os preços para renda variavel
            df = df/df.iloc[0,:]*asset_weights*100
            y = y.append(df)
    y = y[~y.index.duplicated(keep='first')]
    y = y.sum(axis=1)
    y = pd.DataFrame(y).rename({0:ativo},axis=1)
    dados = pd.concat([dados,y],axis=1)
dados = dados.dropna()
dados

[*********************100%***********************]  1 of 1 completed................] 91136 / 91136
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[**

,rf,acao,acao_us,commodities,valor,cripto
2022-05-02,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
2022-05-03,99.847703,99.105231,100.586639,99.432005,100.264926,104.685422
2022-05-04,99.532880,101.554814,102.647053,102.391723,101.249783,98.447090
2022-05-05,100.111584,99.142491,99.438943,100.361488,100.863906,99.070001
2022-05-06,99.621996,97.383528,98.821439,99.989984,101.030924,98.696859
2022-05-09,99.247324,94.382649,96.220022,94.967351,99.568047,85.871638
2022-05-10,99.727929,95.011543,96.338125,94.178945,98.727175,76.676019
2022-05-11,99.309900,95.547355,95.577971,96.116468,99.533492,74.862514
2022-05-12,99.324854,96.682050,95.176514,94.832300,98.007253,75.889724
2022-05-13,99.679857,99.363935,97.384580,97.155527,97.212457,78.493948


In [7]:
# cria coluna com a cotização da carteira total no dataframe "dados"
abas = pd.ExcelFile(f"./pesos/pesos.xlsx").sheet_names # pega a lista de abas na planilha
lista_abas = [aba for aba in abas if aba.startswith('Planilha')]
lista_ativos = list(pd.read_excel('./pesos/pesos.xlsx', sheet_name=lista_abas[-1])["ativo"]) # pega a lista sempre da aba mais recente
dados["carteira"] = 0 #cria a coluna carteira e da o valor de 0 pra tudo
for aba in lista_abas:
    aux = pd.DataFrame(index=dados.index) # dataframe auxiliar pra dar join com o principal depois
    aux["carteira"] = 0
    sheet_peso = pd.read_excel('./pesos/pesos.xlsx', sheet_name=aba)
    pesos = sheet_peso['peso'] # pega os pesos pra multiplicar pela cota de cada ativo
    ini = sheet_peso['data ini'][0]
    fim = sheet_peso['data fim'][0]
    for peso, ativo in zip(pesos, lista_ativos):
        aux["carteira"] += dados[ativo]*peso # multiplica todas as colunas pelo peso correspondente e soma
    aux = aux[ini:fim] # filtra apenas os dados com o peso correto para o período
    dados.update(aux) # atualiza os dados com os dados presentes em aux
dados

,rf,acao,acao_us,commodities,valor,cripto,carteira
2022-05-02,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
2022-05-03,99.847703,99.105231,100.586639,99.432005,100.264926,104.685422,100.256862
2022-05-04,99.532880,101.554814,102.647053,102.391723,101.249783,98.447090,100.497295
2022-05-05,100.111584,99.142491,99.438943,100.361488,100.863906,99.070001,99.755411
2022-05-06,99.621996,97.383528,98.821439,99.989984,101.030924,98.696859,99.014603
2022-05-09,99.247324,94.382649,96.220022,94.967351,99.568047,85.871638,96.238828
2022-05-10,99.727929,95.011543,96.338125,94.178945,98.727175,76.676019,95.610339
2022-05-11,99.309900,95.547355,95.577971,96.116468,99.533492,74.862514,95.403605
2022-05-12,99.324854,96.682050,95.176514,94.832300,98.007253,75.889724,95.577660
2022-05-13,99.679857,99.363935,97.384580,97.155527,97.212457,78.493948,96.953687


In [8]:
# baixa dados pros benchmarks
df = pd.DataFrame({})
ini_date = pd.to_datetime('2022-05-02')
end_date = pd.to_datetime('2023-06-01')

lista = ['^BVSP','^GSPC']
for item in lista:
    df[item] = yf.download(item,ini_date, end_date)['Adj Close']
df = df/df.iloc[0,:]*100
df.dropna(inplace=True)
df.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,^BVSP,^GSPC
Date,,
2022-05-02,100.000000,100.000000
2022-05-03,99.895911,100.483713
2022-05-04,101.598852,103.484400
2022-05-05,98.748113,99.795211
2022-05-06,98.589634,99.228950


In [9]:
# escreve arquivo com os resultados

lista_ativos = list(pd.read_excel('./pesos/pesos.xlsx')["ativo"]) # pega lista_ativos novamente
with pd.ExcelWriter('./resultados/Resultado.xlsx') as writer:
    dados.to_excel(writer, sheet_name="carteira cotizada") # escreve a carteira cotizada no excel
    df.to_excel(writer, sheet_name="benchmarks") # escreve os precos pros benchmarks no excel
    dados = precos_rf() # pega os preços dos titulos de rf
    dados.to_excel(writer, sheet_name="rf") # escreve precos de renda fixa no excel
    lista_ativos.remove("rf")
    # escreve precos pra renda variavel no excel
    for ativo in lista_ativos:
        # pega a lista de abas na planilha que começam com "Planilha"
        abas = pd.ExcelFile(f"./pesos/{ativo}.xlsx").sheet_names 
        lista_abas = [aba for aba in abas if aba.startswith('Planilha')]
        dados = pd.read_excel(f"./pesos/{ativo}.xlsx", 
                              sheet_name=lista_abas[-1]).set_index("ativo") # sheet name da última aba sempre
        dados = precos_rv(dados)  
        dados.to_excel(writer, sheet_name=ativo)
print("Pronto")

[*********************100%***********************]  1 of 1 completed................] 91136 / 91136
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[**